In [1]:
import sys
import json
# TO CHANGE
BASEDIR = "../../.."
sys.path.insert(0, BASEDIR)

In [2]:
from pprint import pprint

In [3]:
from src import PersonalAI, PersonalAIConfig
from src.kg_model import KnowledgeGraphModelConfig
from src.db_drivers.vector_driver.embedders import EmbedderModelConfig

from src.pipelines.qa import QAPipelineConfig
from src.pipelines.qa.kg_reasoning import KnowledgeGraphReasonerConfig
from src.pipelines.qa.query_preprocessing import QueryPreprocessorConfig
from src.pipelines.qa.kg_reasoning.weak_reasoner import WeakKGReasonerConfig
from src.pipelines.qa.kg_reasoning.medium_reasoner import MediumKGReasonerConfig

/home/dzigen/Desktop/Projects/PersonalAI/.pai_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
kg_config = KnowledgeGraphModelConfig(
    nodestree_config=None # Модель дерева вершин строиться не будет
)
EMBEDDER_MODEL_PATH = '../../../models/intfloat/multilingual-e5-base' # PATH TO APPROPRIATE EMBEDDER-MODEL
kg_config.embedders_configs['m-e5-base'] = EmbedderModelConfig(model_name_or_path=EMBEDDER_MODEL_PATH)

In [43]:
pai_config = PersonalAIConfig(
    kg_model_config=kg_config,
    qa_pipeline_config=QAPipelineConfig(
        preprocessor_config=QueryPreprocessorConfig(
            denoising_config=None, # None | by default
            enhancing_config=None, # None | by default
            decomposition_config=None
        ),
        reasoner_config=KnowledgeGraphReasonerConfig(
            reasoner_name='medium', # 'weak' | 'medium'
            reasoner_hyperparameters=MediumKGReasonerConfig()  # WeakKGReasonerConfig() |  MediumKGReasonerConfig()
        )
    )
)

In [54]:
personalai = PersonalAI(pai_config)

No sentence-transformers model found with name ../../../models/intfloat/multilingual-e5-base. Creating a new one with mean pooling.


In [7]:
personalai.mem_pipeline.clear_kv_caches()
personalai.qa_pipeline.clear_kv_caches()

In [46]:
print("mem kv_cache info:")
print(json.dumps(personalai.mem_pipeline.get_cache_stat(), indent=5))
print("mem agent tgen info:")
print(json.dumps(personalai.mem_pipeline.get_agent_tgen_stat(), indent=5))

mem kv_cache info:
{
     "MemPipeline": null,
     "extractor": {
          "LLMExtractor": null,
          "triplets_extraction_solver": 33,
          "thesises_extraction_solver": 33
     },
     "updator": {
          "LLMUpdator": null,
          "replace_simple_solver": 0,
          "replace_hyper_solver": 0
     }
}
mem agent tgen info:
{
     "extractor": {
          "triplets_extraction_solver": {
               "prompt_tokens_amount": {
                    "count": 33,
                    "count_not_null": 33,
                    "min": 846,
                    "max": 996,
                    "median": 877,
                    "mean": 882.485,
                    "std": 28.824,
                    "sum": 29122
               },
               "generated_tokens_amount": {
                    "count": 33,
                    "count_not_null": 33,
                    "min": 20,
                    "max": 538,
                    "median": 67,
                    "mean": 89.182,


In [ ]:
print("qa kv_cache info:")
print(json.dumps(personalai.qa_pipeline.get_cache_stat(), indent=5))
print("qa agent tgen info:")
print(json.dumps(personalai.qa_pipeline.get_agent_tgen_stat(), indent=5))

In [10]:
personalai.kg_model.clear()

In [55]:
print(json.dumps(personalai.kg_model.count_items(detailed=True),indent=5))

{
     "graph_info": {
          "triplets": {
               "simple": 125,
               "hyper": 314,
               "episodic": 455,
               "time": 0
          },
          "nodes": {
               "object": 236,
               "hyper": 97,
               "episodic": 32,
               "time": 0
          }
     },
     "embeddings_info": {
          "nodes": {
               "object": {
                    "nodes_dense": 236,
                    "nodes_sparse_bm25": 236
               },
               "hyper": {
                    "nodes_dense": 97,
                    "nodes_sparse_bm25": 97
               },
               "episodic": {
                    "nodes_dense": 32,
                    "nodes_sparse_bm25": 32
               },
               "time": {
                    "nodes_dense": 0,
                    "nodes_sparse_bm25": 0
               }
          },
          "triplets": {
               "triplets_dense": 250,
               "triplets_sparse_bm25"

In [25]:
TEXT_EXAMPLES = [
    "Проживающие в общежитии студенты имеют право rруглосуточного доступа к месту проживания.",
    "Проживающие в общежитии студенты имеют право обратиться к администрации ФГБУ «МСГ» с заявлением, заверенным заведующим общежития, о размещении в гостевых комнатах общежития родственников (на короткий период пребывания, не менее 2-х суток), родителей - на любой срок (при предоставлении документа, подтверждающего степень родства).",
    "Проживающие в общежитии студенты имеют право пользоваться помещениями для самостоятельных занятий и помещениями культурно-бытового назначения, оборудованием, инвентарем общежития.",
    "Проживающие в общежитии студенты имеют право обращаться к администрации корпуса с просьбами о своевременном ремонте, замене оборудования и инвентаря, вышедшего из строя не по их вине.",
    "Проживающие в общежитии студенты имеют право на переселение из одного помещения в другое в том же корпусе, а также на переселение из одного корпуса в другой при наличии свободных мест, с согласия администрации корпуса.",
    "Проживающие в общежитии студенты имеют право участвовать в формировании и выборах Студенческого совета МСГ и быть избранным в его состав.",
    "Проживающие в общежитии студенты имеют право участвовать (вносить предложения) через Студенческий совет МСГ и отдел по молодежной политике ФГБУ «МСГ» в решении вопросов совершенствования жилищно-бытовых условий, организации воспитательной работы и досуга.",
    "Проживающие в общежитии студенты имеют право принимать участие в общественных, спортивных и культурно-досуговых мероприятиях, организованных администрацией ФГБУ «МСГ» и Студенческим советом МСГ.",
    "Проживающие в общежитии студенты имеют право пользоваться разрешенной бытовой техникой с соблюдением правил техники безопасности и правил пожарной безопасности.",
    "Проживающие в общежитии студенты имеют право бесплатно посещать Межвузовский  учебно-спортивный центр в утвержденное администрацией ФГБУ «МСГ» и согласованное со Студенческим советом МСГ время.",
    "Проживающие в общежитии студенты имеют право бесплатно посещать душевой комплекс с сауной ФГБУ «МСГ» в отведенное администрацией ФГБУ «МСГ» время.",

    "Проживающие в общежитии студенты обязаны ознакомиться с настоящими Правилами.",
    "Проживающие в общежитии студенты обязаны cтрого соблюдать настоящие Правила, правила техники безопасности, правила пожарной безопасности, требования (рекомендации) Рсопотребнадзора.",
    "Проживающие в общежитии студенты обязаны принимать участие в проведении инструкторско-методических занятий по теме: «Организация эвакуации людей из студенческого общежития при возникновении пожара и чрезвычайных ситуациях».",
    "Проживающие в общежитии студенты обязаны при чрезвычайных ситуациях и срабатывании оповещения — покинуть общежитие.",
    "Проживающие в общежитии студенты обязаны выполнять условия договора найма жилого помещения.",
    "Проживающие в общежитии студенты обязаны В установленном порядке и сроки представлять документы для регистрации по месту пребывания, своевременно предоставлять в отдел учета, размещения и регистрации информацию о смене паспортных данных (20 лет — замена паспорта, смена фамилии и др. личные данные).",
    "Проживающие в общежитии студенты обязаны ежегодно представлять администрации корпуса копию справки о прохождении флюорографии (ФЛГ).",
    "Проживающие в общежитии студенты обязаны своевременно в соответствии с условиями договора найма жилого помещения в общежитии вносить плату в установленных размерах за проживание в общежитии и за все виды предоставляемых дополнительных платных услуг.",
    "Проживающие в общежитии студенты обязаны по требованию администрации корпуса предъявлять электронный пропуск установленного образца на право входа в общежитие.",

    "Проживающие в общежитии студенты обязаны обеспечить возможность осмотра жилой комнаты администрацией корпуса в присутствии членов Студенческого совета МСГ или проживающих на этаже студентов с целью контроля санитарного состояния, проверки сохранности имущества, проведения профилактических и других видов работ.",
    "Проживающие в общежитии студенты обязаны не допускать, а в случае обнаружения сообщать администрации корпуса и администрации ФГБУ «МСГ», любые попытки распространения информации или пропаганды сомнительных организаций (террористических), так как это противоречит действующему законодательству РФ.",
    "Проживающие в общежитии студенты обязаны принимать посетителей только в установленное администрацией ФГБУ «МСГ» время, согласно графику посещения гостей, утвержденному директором ФГБУ «МСГ». Время визитов для посторонних посетителей: среда с 17.00 до 21.00, суббота, воскресенье, праздничные дни с 13.00 до 21.00, время визитов для посетителей из ФГБУ «МСГ»: с понедельника по пятницу с 17.00 до 21.00, в субботу, воскресенье, праздничные дни с 13.00 до 21.00.",
    "Проживающие в общежитии студенты обязаны при временном отсутствии более чем на 3 суток (каникулы, практика, стажировка и др.) письменно уведомить администрацию корпуса не менее чем за 2 (два) дня до предполагаемого выбытия.",
    "Проживающие в общежитии студенты обязаныво время пользования помещениями для самостоятельных занятий и культурно-бытового назначения соблюдать тишину и не создавать препятствий другим проживающим в пользовании указанными помещениями.",
    "Проживающие в общежитии студенты обязаныбережно относиться к помещениям, оборудованию и инвентарю.",
    "Проживающие в общежитии студенты обязанынаходясь на кухне, проявлять повьшиенное внимание к работающим бытовым приборам (особенно к газовым плитам) во избежание задымления, возгорания (выключать кипящие чайники, кастрюли, подгорающую пищу и т.п.).",
    "Проживающие в общежитии студенты обязанына основании графика, установленного Старостой этажа, добросовестно нести обязанности по дежурству на кухне.",

    "Проживающие в общежитии студенты обязаныэкономно расходовать электроэнергию, газ и воду.",
    "Проживающие в общежитии студенты обязанысоблюдать чистоту и порядок в жилых помещениях и местах общего пользования, осуществлять влажную уборку жилого помещения не реже 3-х раз в неделю, в соответствии с графиком уборки комнаты.",
    "Проживающие в общежитии студенты обязаныв случае обнаружения неисправностей датчиков пожарной безопасности, электричества, отопительных батарей, инвентаря, оборудования и насекомых сообщить администрации корпуса либо заполнить форму заявки в системе АСУ «Заявки» по адресу: ВИр://арр.т1зе-зрь.га.",
    "Проживающие в общежитии студенты обязаны возмещать причиненный материальный ущерб в соответствии с действующим законодательством и договором найма жилого помещения.",
    "Проживающие в общежитии студенты обязаны соблюдать требования морально-этических норм поведения, поддерживать атмосферу доброжелательности и взаимного уважения по отношению к проживающим, работникам, администрации корпуса, администрации ФГБУ «МСГ» и членам Студенческого совета МСГ."
]

In [13]:
extracted_triplets = []
for i, example in enumerate(TEXT_EXAMPLES):
    print(f"{i+1}. {example}")
    tmp_extracted_triplets, _ = personalai.update_memory(example)
    extracted_triplets += tmp_extracted_triplets
    print("extracted triplets: ", len(tmp_extracted_triplets))

1. Проживающие в общежитии студенты имеют право rруглосуточного доступа к месту проживания.
extracted triplets:  11
2. Проживающие в общежитии студенты имеют право обратиться к администрации ФГБУ «МСГ» с заявлением, заверенным заведующим общежития, о размещении в гостевых комнатах общежития родственников (на короткий период пребывания, не менее 2-х суток), родителей - на любой срок (при предоставлении документа, подтверждающего степень родства).
extracted triplets:  42
3. Проживающие в общежитии студенты имеют право пользоваться помещениями для самостоятельных занятий и помещениями культурно-бытового назначения, оборудованием, инвентарем общежития.
extracted triplets:  11
4. Проживающие в общежитии студенты имеют право обращаться к администрации корпуса с просьбами о своевременном ремонте, замене оборудования и инвентаря, вышедшего из строя не по их вине.
extracted triplets:  25
5. Проживающие в общежитии студенты имеют право на переселение из одного помещения в другое в том же корпусе

In [26]:
print(json.dumps(personalai.kg_model.count_items(detailed=True),indent=5))
personalai.kg_model.check_consistency()

{
     "graph_info": {
          "triplets": {
               "simple": 125,
               "hyper": 314,
               "episodic": 455,
               "time": 0
          },
          "nodes": {
               "object": 236,
               "hyper": 97,
               "episodic": 32,
               "time": 0
          }
     },
     "embeddings_info": {
          "nodes": {
               "object": {
                    "nodes_dense": 236,
                    "nodes_sparse_bm25": 236
               },
               "hyper": {
                    "nodes_dense": 97,
                    "nodes_sparse_bm25": 97
               },
               "episodic": {
                    "nodes_dense": 32,
                    "nodes_sparse_bm25": 32
               },
               "time": {
                    "nodes_dense": 0,
                    "nodes_sparse_bm25": 0
               }
          },
          "triplets": {
               "triplets_dense": 250,
               "triplets_sparse_bm25"

True

In [15]:
print("mem kv_cache info:")
print(json.dumps(personalai.mem_pipeline.get_cache_stat(), indent=5))
print("mem agent tgen info:")
print(json.dumps(personalai.mem_pipeline.get_agent_tgen_stat(), indent=5))

mem kv_cache info:
{
     "MemPipeline": null,
     "extractor": {
          "LLMExtractor": null,
          "triplets_extraction_solver": 33,
          "thesises_extraction_solver": 33
     },
     "updator": {
          "LLMUpdator": null,
          "replace_simple_solver": 0,
          "replace_hyper_solver": 0
     }
}
mem agent tgen info:
{
     "extractor": {
          "triplets_extraction_solver": {
               "prompt_tokens_amount": {
                    "count": 33,
                    "count_not_null": 33,
                    "min": 846,
                    "max": 996,
                    "median": 877,
                    "mean": 882.485,
                    "std": 28.824,
                    "sum": 29122
               },
               "generated_tokens_amount": {
                    "count": 33,
                    "count_not_null": 33,
                    "min": 20,
                    "max": 538,
                    "median": 67,
                    "mean": 89.182,


In [51]:
answer1, rinfo = personalai.answer_question("Какими помещениями разрешено пользоваться студентам, проживающем в общещитии МСГ?")
pprint(answer1)

('На основе предоставленной информации можно сделать вывод, что студентам ФГБУ '
 'МСГ разрешены следующие помещения: межвузовский учебно-спортивный центр, '
 'спортзалы, библиотеки, кафе и другие общественные пространства.')


In [56]:
answer2, rinfo = personalai.answer_question("Разрешено ли студентами, проживающим в общежитии МСГ пользоваться бытовой техникой?")
pprint(answer2)

('Разрешено ли студентам, проживающим в общежитии МСГ, пользоваться бытовой '
 'техникой? Разрешено, но с соблюдением правил техники безопасности, '
 'сообщением об обнаруженных неисправностях и обращением к администрации с '
 'вопросами или жалобами.')


In [41]:
answer3, rinfo = personalai.answer_question("Могут ли проживающие в общежитии МСГ ображаться к администрации с вопросами?")
pprint(answer3)

'Да, проживающие в общежитии МСГ могут ображаться к администрации с вопросами.'


In [50]:
print("qa agent tgen info:")
print(json.dumps(personalai.qa_pipeline.get_agent_tgen_stat(), indent=5))
print("qa kv_cache info:")
print(json.dumps(personalai.qa_pipeline.get_cache_stat(), indent=5))

qa agent tgen info:
{
     "query_preprocessor": {},
     "kg_reasoner": {
          "reasoner": {
               "searchplan_enhancer": {
                    "plan_initialing_solver": {
                         "prompt_tokens_amount": {
                              "count": 3,
                              "count_not_null": 3,
                              "min": 485,
                              "max": 488,
                              "median": 486,
                              "mean": 486.333,
                              "std": 1.247,
                              "sum": 1459
                         },
                         "generated_tokens_amount": {
                              "count": 3,
                              "count_not_null": 3,
                              "min": 119,
                              "max": 149,
                              "median": 149,
                              "mean": 139.0,
                              "std": 14.142,
             

In [52]:
personalai.kg_model.count_items()

{'graph_info': {'triplets': 894, 'nodes': 365},
 'embeddings_info': {'nodes': 365, 'triplets': 250},
 'nodestree_info': None}

In [18]:
#personalai.mem_pipeline.clear_kv_caches()
#personalai.mem_pipeline.clear_agent_tgen_stat()
personalai.qa_pipeline.clear_kv_caches()
personalai.qa_pipeline.clear_agent_tgen_stat()

In [21]:
print("mem kv_cache info:")
print(json.dumps(personalai.mem_pipeline.get_cache_stat(), indent=5))
print("mem agent tgen info:")
print(json.dumps(personalai.mem_pipeline.get_agent_tgen_stat(), indent=5))
print("qa kv_cache info:")
print(json.dumps(personalai.qa_pipeline.get_cache_stat(), indent=5))
print("qa agent tgen info:")
print(json.dumps(personalai.qa_pipeline.get_agent_tgen_stat(), indent=5))

mem kv_cache info:
{
     "MemPipeline": null,
     "extractor": {
          "LLMExtractor": null,
          "triplets_extraction_solver": 0,
          "thesises_extraction_solver": 0
     },
     "updator": {
          "LLMUpdator": null,
          "replace_simple_solver": 0,
          "replace_hyper_solver": 0
     }
}
mem agent tgen info:
{
     "extractor": {
          "triplets_extraction_solver": {
               "prompt_tokens_amount": {
                    "count": 0,
                    "count_not_null": 0,
                    "min": null,
                    "max": null,
                    "median": null,
                    "mean": null,
                    "std": null,
                    "sum": null
               },
               "generated_tokens_amount": {
                    "count": 0,
                    "count_not_null": 0,
                    "min": null,
                    "max": null,
                    "median": null,
                    "mean": null,
      

In [53]:
del personalai